In [7]:
import pandas as pd
import os
import sklearn
import numpy as np
from utils import split_dataset

In [8]:
# Load covariates and convert index to datetime
covariates = pd.read_csv("./data/bdataaug.csv", index_col="Dates", skiprows=5)
covariates.index = pd.to_datetime(covariates.index)

# Load labels and convert index to datetime
labels = pd.read_csv("./data/jkse.csv", skiprows=0, index_col='Date')
labels.index = pd.to_datetime(labels.index)

# Rename column and calculate percentage change
labels = labels.rename(columns={"Close": "JKSE_PRICE"})
labels['PCT_CHANGE_20_JKSE'] = ((labels['JKSE_PRICE'].shift(-20) - labels['JKSE_PRICE']) / labels['JKSE_PRICE']) * 100

# Format the index to the desired format
covariates.index = covariates.index.strftime('%d-%B-%Y')
labels.index = labels.index.strftime('%d-%B-%Y')

Split the dataset per ticker

In [5]:
covlist = split_dataset(covariates)

## Create technical covariates


### PE Band & P/E Ratio  
$$
 \text{P/E Ratio} = \frac{1}{\text{EARN\_YLD}} 
$$

* $PE Band_q $ is the q-th percentile of P/E Ratio in a 60 trading days rolling window

### Mean Average Convergence/Divergence (MACD)

- MACD is the difference of short term $\text{EMA}_{12}$ and long term $\text{EMA}_{26}$ (signal for momentum)
- MACD Signal is the signal line of 9 day EMA of the MACD
- MACD Histogram is the difference between the MACD line and the signal line

In [6]:
colnames = covlist[0].columns.tolist()
for cov in covlist:
    # unify column names
    cov.columns = colnames
for cov in covlist:
    # Volume = Turnover / Close Price
    cov['VOLUME'] = cov['TURNOVER'] / cov['PX_LAST']
    # Calculate percent change * 100
    cov['PCT_CHANGE_20'] = ((cov['PX_LAST'].shift(-20) - cov['PX_LAST']) / cov['PX_LAST']) * 100
    # Ratio 10/30 = mean volume ratio for the last 10 days / mean volume ratio for the last 30 days
    cov['VOL_RATIO_10_20'] = cov['VOLUME'].rolling(window=10).mean() / cov['VOLUME'].rolling(window=20).mean()
    cov['VOL_RATIO_20_40'] = cov['VOLUME'].rolling(window=20).mean() / cov['VOLUME'].rolling(window=40).mean()
    cov['VOL_RATIO_40_80'] = cov['VOLUME'].rolling(window=40).mean() / cov['VOLUME'].rolling(window=80).mean()
    cov['VOL_RATIO_80_120'] = cov['VOLUME'].rolling(window=80).mean() / cov['VOLUME'].rolling(window=120).mean()
    
    # PE Band
    cov['PE_Ratio'] = 1 / cov['EARN_YLD']
    win = 60  # Set the rolling window period
    cov['PE_Band_25'] = cov['PE_Ratio'].rolling(win).quantile(0.25)
    cov['PE_Band_50'] = cov['PE_Ratio'].rolling(win).quantile(0.50)
    cov['PE_Band_75'] = cov['PE_Ratio'].rolling(win).quantile(0.75)

    # Calculate the 12-day EMA of PX_LAST
    ema_12 = cov['PX_LAST'].ewm(span=12, adjust=False).mean()

    # Calculate the 26-day EMA of PX_LAST
    ema_26 = cov['PX_LAST'].ewm(span=26, adjust=False).mean()

    # Calculate MACD
    cov['MACD'] = ema_12 - ema_26

    # Calculate the Signal line (9-day EMA of MACD)
    cov['MACD_Signal'] = cov['MACD'].ewm(span=9, adjust=False).mean()

    # Optionally, you can also calculate the MACD Histogram (the difference between MACD and Signal line)
    cov['MACD_Histogram'] = cov['MACD'] - cov['MACD_Signal']
    
    # Example: Momentum Indicator for various lags
    lags = [10, 20, 30, 60, 120]
    for lag in lags:
        cov[f'MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'TURNOVER_{lag}'] = cov['TURNOVER'].rolling(window=lag).mean()
        cov[f'PX_MOMENTUM_{lag}'] = cov['PX_LAST'] / cov['PX_LAST'].shift(lag)
        cov[f'PX_REVERSAL_{lag}'] = cov['PX_LAST'].shift(lag) / cov['PX_LAST']
        cov[f'VOLATILITY_{lag}'] = cov['PX_LAST'].rolling(window=lag).std()
        cov[f'VOLUME_STD_{lag}'] = cov['VOLUME'].rolling(window=lag).std()

In [ ]:
# Remove duplicate indices if any
labels = labels[~labels.index.duplicated(keep='first')]

for i in range(len(covlist)):
    cov = covlist[i]
    cov = cov[~cov.index.duplicated(keep='first')]

    # Explicitly create a copy of the cov DataFrame to avoid SettingWithCopyWarning
    cov_copy = cov.copy()

    # Align the DataFrames on their indices (dates)
    aligned_df = labels.join(cov_copy[['PCT_CHANGE_20']], how='inner')

    # Calculate the difference and store it in cov_copy DataFrame
    cov_copy.loc[aligned_df.index, 'DELTA_20_CHANGE'] = aligned_df['PCT_CHANGE_20'] - aligned_df['PCT_CHANGE_20_JKSE']

    # Update the original DataFrame in covlist
    covlist[i] = cov_copy


In [ ]:
tickernames = [col[:4] for col in pd.read_csv("./data/bdataaug.csv", skiprows=3).columns if not col.startswith("Unnamed")]

In [ ]:
for i in range(len(covlist)):
    covlist[i]['Ticker'] = tickernames[i]

In [ ]:
covlist[0]

In [ ]:
def unique_index(df, suffix):
    df_copy = df.copy()
    new_index = [f"{date.strftime('%d-%B-%Y')}-{suffix}" for date in df.index]
    df_copy.index = new_index
    return df_copy

Create reindexed covlist + pool to find quantiles + insert index back to other covariates

In [ ]:
covlist_reindex = []
for idx, cov in enumerate(covlist):
    cov['Ticker'] = cov['Ticker'].unique()[0]
    covlist_reindex.append(unique_index(cov, idx))
    
pooled_df = pd.DataFrame()
for i, df in enumerate(covlist_reindex):
    pooled_df = pd.concat([pooled_df, df['DELTA_20_CHANGE']])

pooled_df['DELTA_20_QUINTILES'] = pd.qcut(pooled_df[0], q=10, labels=range(1, 11))

for i, df in enumerate(covlist_reindex):
    df['DELTA_20_QUINTILES'] = pooled_df.loc[df.index, 'DELTA_20_QUINTILES']
    covlist_reindex[i] = df

In [ ]:
for cov in covlist_reindex:
    cov['TOP_5'] = cov['DELTA_20_QUINTILES'].apply(lambda x: 1 if x in [8, 9, 10] else 0)

In [ ]:
covlist_reindex[1]['Ticker']

- covlist reindex of length 342


In [ ]:
import pandas as pd

def rolling_window_label(covlist_reindex, train_window=252, test_window=60):
    labeled_dfs = []

    for cov in covlist_reindex:
        # Create new columns to label the data
        cov['set'] = 'None'
        cov['window'] = 0

        # Iterate over the DataFrame with a rolling window approach in reverse order
        window_num = 1
        for start in range(len(cov) - test_window - train_window, -1, -test_window):
            end_valid = start + test_window
            end_train = end_valid + train_window

            # Label the validation period
            cov.loc[start:end_valid, 'set'] = 'Validation'
            cov.loc[start:end_valid, 'window'] = window_num

            # Label the training period
            cov.loc[end_valid:end_train, 'set'] = 'Training'
            cov.loc[end_valid:end_train, 'window'] = window_num

            window_num += 1

        labeled_dfs.append(cov)

    return labeled_dfs

# Example usage with the adapted function
labeled_covlist = rolling_window_label(covlist_reindex, 252, 60)
labeled_covlist[0].head()  # Display the head of the first labeled DataFrame for inspection

In [ ]:
for i in range(len(covlist_reindex)):
    covlist_reindex[i] = covlist_reindex[i].dropna(axis=0, how='any')
    covlist_reindex[i] = covlist_reindex[i].reset_index(drop=False)

save data

In [ ]:
import pandas as pd

def rolling_window_label(covlist_reindex, train_window=252, test_window=60):
    labeled_dfs = []

    for cov in covlist_reindex:
        # Create new columns to label the data
        cov['set'] = 'None'
        cov['window'] = 0

        # Iterate over the DataFrame with a rolling window approach in reverse order
        window_num = 1
        for start in range(len(cov) - test_window - train_window, -1, -test_window):
            end_valid = start + test_window
            end_train = end_valid + train_window

            # Label the validation period
            cov.iloc[start:end_valid, cov.columns.get_loc('set')] = 'Validation'
            cov.iloc[start:end_valid, cov.columns.get_loc('window')] = window_num

            # Label the training period
            cov.iloc[end_valid:end_train, cov.columns.get_loc('set')] = 'Training'
            cov.iloc[end_valid:end_train, cov.columns.get_loc('window')] = window_num

            window_num += 1

        labeled_dfs.append(cov)

    return labeled_dfs

# Example usage with the adapted function
# Replace 'your_data' with your actual DataFrame list
labeled_covlist = rolling_window_label([covlist_reindex], 252, 60)
print(labeled_covlist[0].head())  # Display the head of the first labeled DataFrame for inspection

In [ ]:
labeled_covlist[0]

In [ ]:
def rolling_window_save(covlist_reindex, train_window=252, test_window=40, valid=False):
    tdf = []
    vdf = []

    for cov in covlist_reindex:
        # Remove specific columns and reset the index but keep the original one
        # cov = cov.drop(['DELTA_20_CHANGE'], axis=1) if valid else cov.drop(['DELTA_20_CHANGE', 'PCT_CHANGE_20'], axis=1)

        # Iterate over the DataFrame with a rolling window approach
        for start in range(0, len(cov) - train_window - test_window + 1, test_window):
            end_train = start + train_window
            end_test = end_train + test_window

            train_df = cov.iloc[start:end_train].reset_index(drop=False)
            test_df = cov.iloc[end_train:end_test].reset_index(drop=False)

            tdf.append(train_df)
            vdf.append(test_df)

    return tdf, vdf

# Example usage
tdf, vdf = rolling_window_save(covlist_reindex, 252, 60)

In [ ]:
len(covlist_reindex)

In [ ]:
covlist_reindex[4]

In [ ]:
for idx, vdfe in enumerate(covlist_reindex):
    i = 0
    if len(vdfe) != 0:
        print(idx, vdfe.loc[1, "Ticker"])
    else:
        i += 1
print(f"missing {i} tickers")

In [ ]:
len(vdf)

In [ ]:
for v in vdf:
    v.index = v['index']
    v.drop(['level_0', 'index', 'DELTA_20_CHANGE'], axis=1, inplace=True)

In [ ]:
for t in tdf:
    t.index = t['index']
    t.drop(['level_0', 'index', 'DELTA_20_CHANGE', 'PCT_CHANGE_20'], axis=1, inplace=True)

In [ ]:
train = pd.concat(tdf, axis=0)
valid = pd.concat(vdf, axis=0)

In [ ]:
x_valid = valid.drop(['PCT_CHANGE_20', 'DELTA_20_QUINTILES', 'Ticker', 'TOP_5'], axis=1)
y_valid = valid[['TOP_5']]

In [ ]:
x_train = train.drop(['DELTA_20_QUINTILES', 'Ticker', 'TOP_5'], axis=1)
y_train = train[['TOP_5']]

In [ ]:
import joblib

joblib.dump(x_valid, './models/yuge70_xvalid.joblib')
joblib.dump(y_valid, './models/yuge70_yvalid.joblib')
joblib.dump(valid, './models/yuge70_validfull.joblib')

In [ ]:
x_valid.to_csv('./data/yuge70_xvalid.csv', index=True)
y_valid.to_csv('./data/yuge70_yvalid.csv', index=True)
x_train.to_csv('./data/yuge70_xtrain.csv', index=True)
y_train.to_csv('./data/yuge70_ytrain.csv', index=True)


In [ ]:
valid = pd.concat(, axis=0)
valid.index = valid['index']
valid.drop('index', axis=1, inplace=True)

In [ ]:
valid.loc[:, ['PX_LAST', "VOLATILITY_20", 'Ticker', 'DELTA_20_CHANGE', "DELTA_20_QUINTILES", "TOP_5", "PCT_CHANGE_20"]]

In [ ]:
folder_name = 'bdataaug'
os.makedirs(folder_name, exist_ok=True)
for i in range(len(tdf)):
    tdf[i].to_csv(f'{folder_name}/train_{i}.csv', index=False)
    vdf[i].to_csv(f'{folder_name}/valid_{i}_pred.csv', index=False)

In [ ]:
valid.to_csv(f'{folder_name}/valid_pred_all.csv', index=True)